# Eurovision Turing Data Story

> Exploring (non-musical) factors contributing to Eurovision votes in the past 25 years.

- toc: false
- categories: [data wrangling, data visualisation, bayesian modeling, random forest, eurovision]
- author(s): Ed Chapman, Katriona Goldmann, Radka Jersakova, David Llewellyn-Jones, Joe Palmer, Camila Rangel Smith, Martin Stoffel, Jonathan Yong
- image: TODO

**Authors**
 - Ed Chapman
 - Katriona Goldmann
 - Radka Jersakova
 - David Llewellyn-Jones
 - Joe Palmer
 - Camila Rangel Smith
 - Martin Stoffel
 - Jonathan Yong
 
 **Reviewers:**
- Reviewer 1
- Reviewer 2

# Introduction

The Eurovision Song Contest, or *Eurovision* for short, is an annual competition featuring (mostly) European countries. It is in fact the longest running international televised music competitions, having started in 1951 as what was then seen as a technical experiment in transnational live broadcasting. The competition has continued every year since &mdash; except in 2020 when it was cancelled, [much to Iceland's chagrin](https://youtu.be/1HU7ocv3S2o), due to the pandemic &dash; and has grown in terms of number of countries participating, musical variety, visual flair and (some might say) fantastical preposterousness.

Despite the name, the Eurovision Song Content is neither about Europe, nor Singing. According to the Eurovision website, the European Broadcasting Union which runs the contest is made up of "56 countries and an additional 31 Associates in Asia, Africa, Australasia and the Americas". The contest itself is billed as a *songwriting* competition.

There are [strict requirements](https://eurovision.tv/about/rules) that songs and performers must comply with. Songs must be original; under three minutes long; sung live without lip syncing and with no live plugging instruments.

Typically the winning country is expected to host the technically challenging and costly event the following year (a 'prize' that [allegedly led to](https://www.irelandbeforeyoudie.com/why-ireland-stopped-winning-eurovision/) Ireland entering sub-par acts for at least a decade in a deliberate attempt to avoid winning). As many readers will no-doubt be aware, in 2022 the competition was won by Kalush Orchestra from Ukraine, with their song [Stefania](https://youtu.be/F1fl60ypdLs). This would ordinarily mean the 2023 event would be held in Ukraine, however this was deemed unsafe in light of the Russian invasion. The UK (the 2022 runners-up) therefore stepped in, meaning that this year's contest was held in Liverpool, on 9–13 May 2023.

The seventy-year history of the event means there is now a large body of data about it to work with, as well as a large number of great songs to listen to. As you work through this Turing Data Story, we strongly recommend you listen to some of the great Eurovision masterpieces as your backing track, from the likes of [Celine Dion](https://youtu.be/w6b7BHGkKQA), [Bucks Fizz](https://youtu.be/h4-lKMGII_k), [Lordi](https://youtu.be/gAh9NRGNhUU), [Loreen](https://youtu.be/Pfo-8z86x80) and of course [Abba](https://youtu.be/Vp1_OKawHYw).

<center>
    <img alt="meme comparing what I think I look like discussing eurovision: two men casually talking to sofa; to what I actually look like: Charlie Kelly looking wild-eyed in front of a peg board full of conspiracy connections" src="eurovision_meme.jpeg" />
</center>

In the current format, participating countries first take part in a semifinal; the top 10 from each semifinal qualify for the finals.
The host country, as well as the "Big Five" (France, Germany, Italy, Spain, and the UK, which make greater financial contributions), directly qualify for the finals.

Our aim with this Story has been to try to uncover some of the hidden mysteries of Eurovision voting, and also to make some predictions &mdash; based purely on this data &mdash; for which countries are likely to fare well, and which less well, at this year's contest.

We have pooled data from a number of different sources with the aim of understanding trends in the contest voting patterns. We have also tried to make the data as accessible as possible. The curated data frame is available in the `data/df_main.csv` file, and the code to create this dataset is in `data.ipynb`.

Our goal was to use this dataset to predict the winners of the [2023 event in Liverpool](https://eurovision.tv/event/liverpool-2023). Now that the event has taken place we've left this analysis as-is, so that you can compare our predictions with the actual results. But we've also now been able to add some post-match analysis to reflect on how our models did so that we can be even more accurate next year.

In this notebook we'll go into some detail about all of our data collection, analysis and results. If you're here for instant gratification and just want to know how we did, check out our summary on the [Turing Institute Blog](https://www.turing.ac.uk/blog/can-data-science-help-us-predict-winner-eurovision-2023). But if you want to know all the gory data-sciency details, then read on!

## Setup

We begin by importing packages and setting up any configuration needed.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from pathlib import Path
import seaborn as sns

sns.set_theme(style="whitegrid")

if not Path("../../plots/").exists():
    Path("../../plots/").mkdir()

# Data

In order to explore the characteristics of Eurovision, with the aim of making predictions for the 2023 event, we first need to collect historical data about contests prior to 2023. We will obviously need to know what scores were assigned to which acts in previous years, but we also want to explore other factors that might potentially affect the success or otherwise of the performances.

It has long [been claimed](https://www.datalytyx.com/eurovision-song-contest-regression-analysis-highlights-the-voting-patterns/) that there is a [measurable bias](https://www.tandfonline.com/doi/full/10.1080/02664763.2014.909792) in relation to which countries vote for which other countries at Eurovision. We should therefore gather data that exhibits these biases. We also consider other factors that may affect voting patterns, such as the gender of the performer or language of the song.

The Eurovision Song Contest has undergone several changes to its voting system since it began in 1956. These changes were made to increase transparency and reduce the impact of regional bloc voting, while still allowing for a fair and entertaining competition.

In the early years, the winning country was selected by a jury of experts from each participating country. In 1975, a new system was introduced that combined jury voting with telephone voting by viewers at home. Over time, the weight of the jury and viewer votes has shifted, with viewer votes becoming more influential. In 2016, the voting system was revised again, with separate scores given for jury and viewer votes, and a new system for calculating the final scores. 

In order to model uniform voting scores over time, we limited our analysis to contests from 1998 onwards, where the jury and televoting scores are equally weighted.

In the following sections we'll describe our data-wrangling efforts in some depth. If you're more interested in the analysis, then you can safely skip forwards to the [Covariate visualisation section](#Covariate-visualisation) without losing the narrative.

## Voting Scores

Our voting data was pulled from the following sources: 

- From the [Eurovision Song Contest Scores Kaggle dataset](https://www.kaggle.com/datasets/datagraver/eurovision-song-contest-scores-19752019), for the years 1998–2019. 
- The 2020 contest was cancelled due to Covid. 
- The [2021](https://en.wikipedia.org/wiki/Eurovision_Song_Contest_2021#Final_2) and [2022](https://en.wikipedia.org/wiki/Eurovision_Song_Contest_2022#Final_2) contest scores were scraped from Wikipedia.

The jury votes have in the past been replaced with a substitute aggregate due to potential bias or 'irregularities'. In this case, new jury scores were calculated ["based on the results of other countries with similar voting records"](https://eurovision.tv/mediacentre/release/ebu-statement-voting-during-2022-shows). However, this is not very transparent and [not captured by our models](https://eurovisionworld.com/esc/here-is-the-proof-of-the-eurovision-voting-scandal-six-juries-cheated-and-voted-for-each-other).

Unlike in earlier years, since 2016 the jury and televoting scores have been reported separately. Simply summing these would lead to scores that are unusually large compared to previous years. In order to bring this in line, both votes were summed and rescaled by mapping the highest result to 12 points, second-highest to 10 points, and so on.

Let's load in the Kaggle data (we host our own copy to avoid having to negotiate with Kaggle's authentication wall).

In [ ]:
## Read in data from the Kaggle dataset
votes_1975_2019 = pd.read_excel('https://github.com/KatrionaGoldmann/Eurovision_TDS/raw/main/data/eurovision_song_contest_1975_2019.xlsx', engine='openpyxl')
print('Number of entries: {}'.format(len(votes_1975_2019)))
votes_1975_2019.head()

The dataset requires some cleaning up before we can properly make use of it. Some of this is due to errors that have crept into the dataset such as incorrect spellings of country names, some is due to geopolitical changes during the period we're considering (we have mapped all entries by the entity 'Serbia & Montenegro' to Yugoslavia), and some is needed to get the dataset structured appropriately for our analysis.

In [ ]:
# Clean up the column names
votes_1975_2019.columns = [c.strip().lower().replace(' ', '_') for c in votes_1975_2019.columns.values.tolist()]

# Select only finals votes, and only 1998 onwards (inclusive)
votes_1998_2019 = votes_1975_2019[(votes_1975_2019['(semi-)_final'] == 'f') & (votes_1975_2019['year'] >= 1998)]

# Drop unnecessary columns
votes_1998_2019 = votes_1998_2019[["year", "from_country", "to_country", "points", "jury_or_televoting"]]

# Clean up country names
def standardise_country(c):
    replacements = [('-', ' '), ('&', 'and'), ('Netherands', 'Netherlands'),
                    # FYR Macedonia was formally renamed as North Macedonia in 2019
                    ('F.Y.R. Macedonia', 'North Macedonia'), 
                    ('Russia', 'Russian Federation'), 
                    ('The Netherlands', 'Netherlands'), 
                    ('Czech Republic', 'Czechia'),
                    # Yugoslavia dissolved in 2002; most of it became 'Serbia and Montenegro', until 2006, when Serbia and Montenegro split ways.
                    ('Serbia and Montenegro', 'yugoslavia'),
                    ('moldova', 'moldova, republic of')]
    for r in replacements:
        c = c.replace(r[0], r[1])
    return c.lower()

votes_1998_2019[['from_country', 'to_country']] = votes_1998_2019[['from_country', 'to_country']].applymap(standardise_country)

# Drop columns which correspond to the same vote (there are two Belarus -> Russia in 2019, for example)
votes_1998_2019 = votes_1998_2019.drop_duplicates(subset=['year', 'from_country', 'to_country', 'jury_or_televoting'])

# Drop Lithuania in 2003 (they didn't participate - we don't know why it's still in the dataset)
votes_1998_2019 = votes_1998_2019[~((votes_1998_2019['to_country'] == 'lithuania') & (votes_1998_2019['year'] == 2003))]

# Drop "votes" from one country to itself
votes_1998_2019 = votes_1998_2019[votes_1998_2019['from_country'] != votes_1998_2019['to_country']]

votes_1998_2019.sample(n=10)

The Kaggle dataset runs up to 2019, but we need data for 2021 and 2022 as well (recall that there was no contest in 2020). We collect these data from the tables in the Wikipedia pages for the [2021](https://en.wikipedia.org/wiki/Eurovision_Song_Contest_2021#Final_2) and [2022](https://en.wikipedia.org/wiki/Eurovision_Song_Contest_2022#Final_2) contests respectively.

The process of downloading and cleaning up this data is rather long winded, so we're going to skip past that here and instead refer to the separate [data collection notebook](https://github.com/KatrionaGoldmann/Eurovision_TDS/blob/story_notebook/eurovision/notebooks/data.ipynb) that we've put together for the inquisitve.

So we'll short-circuit this by grabbing the output from the data collection notebook. Or, in the immortal words of the Blue Peter team, here's one I made earlier:

In [ ]:
votes = pd.read_csv('../../data/votes.csv')
votes.head()

## Performance language

The next step is to collect together the languages used in the songs. We're particularly interested in whether the song is sung in the language of the performing country or some other language. Anecdotally there has been a tendency for songs to be sung in English if not a country's native language, and so we also keep track of whether a song includes English lyrics as well.

When collecting the data, we have to bear in mind that songs are often sung in *more than one* language. The fields we've chosen to populate are therefore:
 - *whether the song contains English lyrics*,
 - *whether it contains non-English lyrics*,
 - *whether it contains the performing country's official language*,
 - *whether it contains the voting country's official language*, and
 - *whether it contains multiple languages*.

We retreived information about the language of each performance from a dataset [available from Kaggle](https://www.kaggle.com/datasets/minitree/eurovision-song-lyrics?select=eurovision-lyrics-2022.json). This provides us with half of our needs; the other half relates to which languages are the official languages of each country.

For the latter we scrape the list of official languages available [on Wikipedia](https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory).

Again, the data needed considerable cleaning and structuring to suit our needs. We'll skip the code for doing this for brevity and refer the interested reader to our data collection notebook.

Let's load the results saved out from that notebook here so we can see how things are progressing.

In [ ]:
df_VL = pd.read_csv('../../data/language.csv')
df_VL.head()

## Performer gender

To establish the gender of the various performers we capture the *'gender'* property available from [Wikidata](https://www.wikidata.org/wiki/Wikidata:List_of_properties), using code that's based on Sam Van Stroud's [wikipeople script](https://github.com/samvanstroud/wikipeople/blob/master/wikipeople/wikipeople.py).

Rather than referring directly to our [data collection notebook](https://github.com/KatrionaGoldmann/Eurovision_TDS/blob/story_notebook/eurovision/notebooks/data.ipynb) we'll briefly go through the code here, since it demonstrates some nice techniques.

Establishing gender isn't straightforward in all cases. For example, many Eurovision entries are group performances. For these we detect the *'instance of'* Wikidata property, checking whether it's equal to one of *group*, *duo*, *trio*, *music*, *band* or *ensemble*. There are also non-binary performers to consider.

Searching Wikidata with a name often generates multiple hits, so we try to restrict our search to performers tagged as being musicians and tagged as having performed at Eurovision. Even with this restriction we don't always get a unique hit, or any hit at all. Some artists simply don't exist in the database, or are missing these tags. Out of the total 600 performers we are left having to manually assign genders to 55 of them.

We've collapsed the code that collects the properties from Wikidata. Feel free to take a look, but the more interesting part follows below.

In [ ]:
import aiohttp
import asyncio

async def get_property(session, concept_id, property_id):
    """Async reimplementation of wikipeople.get_property
    https://github.com/samvanstroud/wikipeople/blob/master/wikipeople/wikipeople.py

    session is an aiohttp ClientSession instance.
    concept_id refers to a person or a group, and can be obtained using the get_concept_id function.
    property_id refers to a certain property of the given entity referred to by the concept_id.

    Returns None if any of this can't be found for whatever reason.

    e.g. "Q219655" is the concept_id for Carey Mulligan; "P21" is the property_id for gender.
    So we have that get_property(session, "Q219655", "P21") -> "female".
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbgetclaims",
        "entity": concept_id,
        "property": property_id,
        "language": "en",
        "format": "json",
    }
    async with session.get(url, params=params) as resp:
        try:
            res = await resp.json()
        except Exception as e:
            print(resp)
            raise e

    if property_id not in res["claims"]:
        return None
    # This gives yet another 'id', and we then need to perform yet another HTTP
    # request to find the actual *label* that this corresponds to.
    else:
        id = None
        for prop in res["claims"][property_id]:
            try:
                id = prop["mainsnak"]["datavalue"]["value"]["id"]
            except:
                continue

        if id is None:
            return None
        else:
            new_params = {
                "action": "wbgetentities",
                "ids": id,
                "languages": "en",
                "format": "json",
                "props": "labels",
            }
            async with session.get(url, params=new_params) as resp:
                try:
                    res = await resp.json()
                except Exception as e:
                    print(resp)
                    raise e
            try:
                return res["entities"][id]["labels"]["en"]["value"]
            except:
                return None


async def get_concept_id(session, page_name):
    """
    Get the concept_id corresponding to a particular Wikipedia page. For some odd reason, some Wikipedia
    pages don't have concept IDs. In such a case, we return None.

    e.g. get_concept_id(session, "Carey Mulligan") -> "Q219655"
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "search": page_name,
        "language": "en",
        "format": "json",
    }
    music_markers = [
        "singer",
        "artist",
        "musician",
        "music",
        "band",
        "group",
        "duo",
        "ensemble",
    ]

    async with session.get(url, params=params) as resp:
        # Titles of WP pages that match the search query.
        json = await resp.json()

    result = json["search"]

    if len(result) == 0:
        # Couldn't find a concept id for the person/group
        return None

    # By default, choose the first result from the list
    target = 0
    # But check the other results to see if any of them are musicians (as
    # indicated by the markers) and Eurovision contestants
    for i, res in enumerate(result):
        if "description" in res["display"]:
            description = res["display"]["description"]["value"]
            if any(markers in description for markers in music_markers):
                concept_id = res["id"]
                contestant_in = await get_property(session, concept_id, "P1344")
                if contestant_in is not None and "Eurovision" in contestant_in:
                    target = i
    # Return the concept ID of the result found
    return result[target]["id"]


async def lookup_gender(session, page_name):
    """Find gender of a performing act, using the name associated with their
    Wikipedia page. Returns None if could not be found.
    """
    concept_id = await get_concept_id(session, page_name)
    if concept_id is None:
        return None

    gender = await get_property(session, concept_id, "P21")
    instance = await get_property(session, concept_id, "P31")
    if gender is None and instance is None:
        return None
    elif gender is None and instance is not None:
        group_checks = ["group", "duo", "trio", "music", "band", "ensemble"]
        if any(x in instance for x in group_checks):
            return "group"
    else:
        return gender


async def get_pages(session, name):
    """Obtain a list of Wikipedia pages obtained by searching for a name."""
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "opensearch",
        "namespace": "0",
        "search": name,
        "limit": "10000",
        "format": "json",
    }
    async with session.get(url, params=params) as resp:
        # Titles of WP pages that match the search query.
        json = await resp.json()
    return json[1]


async def get_artist_gender(session, name):
    gender = None
    # Get the WP page for this person/group
    pages = await get_pages(session, name)
    # If there's one, try to get their gender from the first page
    if len(pages) > 0:
        gender = await lookup_gender(session, pages[0])
    # Finally we use some heuristics to cover some edge cases
    if gender is None:
        if "&" in name or "feat." in name:
            return "group"

    return gender

In the code below we make use of the collapsed library functions. The code pulls the list of artists from the data we already collected above.  We use an async function to access Wikidata in batches of forty artists. This allows us to make multiple HTTP requests simultaneously and greatly decreases the overall time needed to work through the list.

Having extracted as much as possible from Wikidata the code then performs the manual fixes to complete the missing entries, and to ensure transgender entries are regarded as simply *male* or *female* as appropriate. Finally the results are merged in with our existing data.

In [ ]:
all_performers = df_VL['Artist'].unique().tolist()
n_performers = len(all_performers)
MAX_CONCURRENT = 40   # To stop Wikipedia from complaining about 'too many requests'
USER_AGENT = 'Eurovision study @ The Alan Turing Institute mailto:jyong@turing.ac.uk'

async def get_all_genders():
    genders = []
    print(f'We need to fetch the genders of {n_performers} performers, in batches of {MAX_CONCURRENT}. Hold tight...')
    async with aiohttp.ClientSession(headers={'User-Agent': USER_AGENT}) as session:
        start = 0
        end = MAX_CONCURRENT
        while start < n_performers:
            print(f'Getting genders for performers #{start + 1} to #{end}... ', end='')
            batch_tasks = asyncio.gather(*[get_artist_gender(session, p) for p in all_performers[start:end]])
            batch_genders = await batch_tasks
            print(f'Got {len(batch_genders)} results, {len([g for g in batch_genders if g is None])} of which were None.')
            genders = genders + batch_genders
            start = end
            end = min(end + MAX_CONCURRENT, n_performers)
            await asyncio.sleep(1.5)   # Put a pause between batches to avoid being timed out

    assert len(genders) == n_performers  # Check for off-by-one errors
    print('Finished downloading gender data.')
    return dict(zip(all_performers, genders))

gender_dict = await get_all_genders()

# Manually assign missing entries (the Nones).
male = ['Michael Hajiyanni', 'Charlie', 'Tüzmen', 'Mietek Szcześniak', 'Olexandr', 'Max', 'Brinck',
        'Sakis Rouvas (2)', 'Gianluca', 'Frans', 'Chingiz', 'Mahmood', 'Serhat (2)', 'Miki', 'Stefan']
female = ['Gunvor', 'Selma', 'Charlotte Nilsson (Perrelli)', 'Karolina', 'Laura', 'Rosa', 'Lou', 'Nicola',
        'Karmen', 'Sanda', 'Ortal', 'Gracia', 'Chiara (2)', 'Hanna', 'Chiara (3)', 'Elena', 'Lena (2)',
        'Birgit', 'Samra', 'ZAA Sanja Vučić', 'Anja', 'Alma', 'Netta', 'Michela', 'Efendi', 'Victoria',
        'Destiny', 'Amanda Georgiadi Tenfjord', 'MARO']
group = ['Eden', 'Voice', 'Taxi', 'One', 'Prime Minister', 'Fame', 'Regina (band)', 'ESDM',
        'Tolmachevy Sisters', 'Minus One', 'AWS']
for p in male:
    gender_dict[p] = "male"
for p in female:
    gender_dict[p] = "female"
for p in group:
    gender_dict[p] = "group"

# Wikipedia needs to learn that 'trans woman' is 'female'.
for k, v in gender_dict.items():
    if v == 'trans woman':
        gender_dict[k] = 'female'

# Add gender to the dataframe.
df_VLG = df_VL.copy()
df_VLG['gender'] = df_VLG['Artist'].map(gender_dict)
df_VLG.head()

## Migration data

Migration data were gathered to report on the estimated migration betweeen each voting–performing country pair of countries that have previously competed. 

Data were collected from two sources. First we took migration data from [Our World in Data](https://ourworldindata.org/migration), under the 'Explore data on where people migrate from and to' section. The underlying data originate from the UN. The data show the total number of immigrants in each country split by country of origin in the years 1990–2020, recorded at intervals of every 5 years.

We clean up the data so that the country names match those we're already using and filter on countries that participate in Eurovision.

In [ ]:
migration = (pd.read_csv('../../data/migration-flows.csv')
    .pipe(pd.melt, id_vars=['Country', 'Year'], var_name='Migration', value_name='Count')  # to long format
    .loc[lambda x: x['Migration'].str.contains('Emigrants')]                               # filter for emigrant rows
    .pipe(lambda x: x.rename(columns = {col: col.lower() for col in x.columns}))           # lowercase column names                                                         
    .assign(migration = lambda x: x.migration.str.replace('Emigrants from ', ''))          # filter for emigrant rows                          
    .rename(columns={'migration': 'emigrated_from', 'country': 'emigrated_to'})            # boil down to country name
    .query('count >= 0')                                                                   # negative counts are just total emigrants from country
    .pipe(lambda x: x.assign(count = x['count'].astype(int)))                              # convert count to int     
)

# Clean up country names
for ft in ['from', 'to']:
    migration[f'emigrated_{ft}'] = migration[f'emigrated_{ft}'].str.lower()
    migration.loc[migration[f'emigrated_{ft}'] == 'moldova', f'emigrated_{ft}'] = 'moldova, republic of'
    migration.loc[migration[f'emigrated_{ft}'] == 'russia', f'emigrated_{ft}'] = 'russian federation'

# Remove non-Eurovision countries
ev_countries = set(df_VLG['from_country']).union(set(df_VLG['to_country']))
migration = migration[(migration['emigrated_to'].isin(ev_countries)) & (migration['emigrated_from'].isin(ev_countries))]

# Add in country codes
def get_country_codes(name):
    return df_VLG[df_VLG['from_country'] == name].iloc[0][['from_code2', 'from_code3']]

for ft in ['from', 'to']:
    migration[f'emigrated_{ft}_code2'], migration[f'emigrated_{ft}_code3'] = zip(*migration[f'emigrated_{ft}'].map(get_country_codes))

We note that we're able to get data for all of the countries except Yugoslavia which is missing from the dataset.

In [ ]:
migration_countries = set(migration['emigrated_to']).union(set(migration['emigrated_from']))
print('Eurovision countries without data: {}'.format(''.join(ev_countries - migration_countries)))  # No data for Yugoslavia.

In order to provide a fairer comparison we convert absolute migration numbers into proportions of the overall population size of each country. For this we also need country population data, which we collected from the [World Bank](https://data.worldbank.org/indicator/SP.POP.TOTL?end=2021&start=2021&view=map).

In [ ]:
pop_size = (pd.read_csv('../../data/pop_sizes.csv')
           .iloc[:, 3:]
           .rename(columns=lambda x: x.lower().replace(' ', '_'))
           .pipe(pd.melt, id_vars=['country_code'], var_name='year', value_name='population')
           .assign(year=lambda x: x['year'].apply(lambda y: y.split('_')[0]))
           .assign(year=lambda x: x['year'].astype(int))
           .rename(columns={'country_code': 'code3'})
           .dropna()
           .assign(population=lambda x: pd.to_numeric(x['population'], errors='coerce'))
)

migration_and_pop = (migration.merge(pop_size, left_on=['year', 'emigrated_to_code3'], right_on=['year', 'code3'], how='left')
                    .rename(columns={'population': 'population_to'})
                    .assign(prop_emigrants=lambda x: x['count'] / x['population_to'])
                    )

The UN migration data are recorded in five year intervals, so for each year of Eurovision data we assign the most recent recording. In essence, we end up repeating each row of the migration data five times, once for each of the following years.

In [ ]:
migration_and_pop['migration_pop_year'] = migration_and_pop['year']

total_migration_and_pop = migration_and_pop.copy()
for i in range(1, 5):
    next_migration_and_pop = migration_and_pop.copy()
    next_migration_and_pop['year'] = next_migration_and_pop['year'] + i
    total_migration_and_pop = pd.concat([total_migration_and_pop, next_migration_and_pop], ignore_index=True)

total_migration_and_pop = total_migration_and_pop.sort_values(by=["emigrated_from", "emigrated_to", "year"])
total_migration_and_pop = total_migration_and_pop[['emigrated_from_code2', 'emigrated_to_code2', 'year', 'count', 'population_to', 'prop_emigrants', 'migration_pop_year']]

# Output a sample of the data including only some of the more relevant columns
total_migration_and_pop[['emigrated_to_code2', 'emigrated_from_code2', 'year', 'population_to', 'prop_emigrants']].sample(5)

Finally we integrate this with our existing dataset. We'll skip this step for brevity and pull the data in direction from the [data collection notebook](https://github.com/KatrionaGoldmann/Eurovision_TDS/blob/story_notebook/eurovision/notebooks/data.ipynb) as before instead.

The result is our original dataframe with additional columns with migration and population data. Note the caveat that we're not working with perfect data here: we don't have data for Yugoslavia, and our data are repeated for the unrecorded years between the five year intervals.

In [ ]:
df_VLGM = pd.read_csv('../../data/migration.csv')

## Competition winners

Traditionally, the winner of each Eurovision contest becomes the host country for the following year's contest, which is considered to be an expensive operation. This has led to a recurring argument that many countries do not want to win, and therefore put in less effort. At the same time, it has previously been proposed that the number of years since a country's last win influences how likely they are to do well due to how competitive their performance is.

In order to investigate this, we calculated how long each performing country had gone without a win in Eurovision. To achieve this, the previous contest winners were gathered from [Wikipedia](https://en.wikipedia.org/wiki/List_of_Eurovision_Song_Contest_winners). 

Finally we integrate this with our existing dataset. We'll skip this step for brevity and pull the data in direction from the [data collection notebook](https://github.com/KatrionaGoldmann/Eurovision_TDS/blob/story_notebook/eurovision/notebooks/data.ipynb) as before instead.

In [ ]:
df_VLGMC = pd.read_csv('../../data/comps_without_win.csv')

# Output a sample of the data for 2022, including only some of the more relevant columns
df_VLGMC[df_VLGMC['year'] == 2022][['to_country', 'comps_without_win']].sample(5)

## Shared border

It's been suggested that neighbouring countries may be *more* &mdash; or contrariwise *less* &mdash; likely to vote for one another than countries further away. Is this really the case, and if so, which is it? Or is it just a case of commentator cynicism and *post-hoc* rationalisation?

We decided to find out by adding shared borders between countries into our model. We collected these country border data from [GeoDataSource](https://github.com/geodatasource/country-borders/). This only includes land borders, so sea borders are not considered. 

For the detailed process of cleaning up and merging the data into our dataset, see the [data collection notebook](https://github.com/KatrionaGoldmann/Eurovision_TDS/blob/story_notebook/eurovision/notebooks/data.ipynb).

In [ ]:
# This represents our final dataset for the historical data
df = df_VLGMCB = pd.read_csv('../../data/df_main.csv')

# Output a sample of the data including only some of the more relevant columns
df_VLGMCB[['from_country', 'to_country', 'has_border']].sample(5)

In [ ]:
print('Years covered: {} to {}'.format(df['year'].min(), df['year'].max()))

## 2023 performers

Existing data sources have provided data covering all previous years up to 2022. As we noted above, we restrict our data to 1998 onwards so that we can focus on years in which both public and jury voting was used.

Our model will make predictions based on the parameters we've provided &mdash; things like the country a performer is from, their gender and the language of the song &mdash; which means that in order to make predictions for 2023, we'll need to know the same characteristics for the 2023 entries. Our model can then use these characteristics in combination with the historical data, to predict the ranking, or scores, of the performances for 2023.

The final step in the collection of our data was therefore to find information about the upcoming performances in 2023. These data were gathered from [Wikipedia](https://en.wikipedia.org/wiki/Eurovision_Song_Contest_2023). Some of the parameters for which we weren't able to obtain more recent data &mdash; such as migration numbers &mdash; we repeat the entries from previous years.

The code for pulling this data together repeats many of the steps we described previously for the historical data, and so we won't go through the details again again here. You can see the full process in the [data collection notebook](https://github.com/KatrionaGoldmann/Eurovision_TDS/blob/story_notebook/eurovision/notebooks/data.ipynb). As usual we will pull in the results as they were output from that notebook and just review a sample.

This gives us all of the data we need. In the next section we'll examine this data and find out what it tells us about Eurovision entries and voting patterns.

In [ ]:
future = pd.read_csv('../../data/df_2023.csv')

# Output a sample of the data including only some of the more relevant columns
future[['to_country', 'gender', 'Contains_English', 'Contains_NonEnglish', 'comps_without_win']].sample(5)

# Exploratory Visualisations

Before we start modelling, we explore patterns in the data through summary statistics and visualisations. The aim here is to gain an intuition about trends, relationships between variables, potential quirks and outliers and voting patterns between countries.

## Best performing countries

Let's first have a look at the winners in each year from 1998 to 2022:

In [ ]:
winners = df.loc[df['rank'] == 1, ['to_country', 'total_points', 'rank', 'to_code2', 'year']]
winners = winners.drop_duplicates()

winners

Winning Eurovision is hard! Only four countries have managed to win more than once since 1998, and only Ukraine 🇺🇦 and Sweden 🇸🇪  won three times.

In [ ]:
multiple_winners = winners['to_country'].value_counts().loc[lambda x: x > 1]
multiple_winners

Winning is one thing, but which countries perform best _on average_? To make this statistic a bit more robust, we only look at countries that have made the finals at least five times within the timeframe we're considering (see the `count` variable).
Over this period, Italy 🇮🇹 has the highest average score, closely followed by Bulgaria 🇧🇬.

In [ ]:
highest_scorers = (df.loc[:, ['to_country', 'to_code2', 'total_points', 'year']]
                   .drop_duplicates()
                   .groupby('to_country')['total_points']
                   .agg(['count', 'mean'])
                   .query('count >= 5')
                   .round({'mean': 2})
                   .sort_values(by='mean', ascending=False)
                   .head()
                   )
highest_scorers

While summary statistics like the average can be informative, it would be really interesting to see how well a country is doing over the years in a bit more detail.
Are the top performing countries always much better than average, or do a few great perfomances lift up the average scores of these countries?

To visualise this for different countries, we create a function to plot the scores of a country over time and mark specific years where the country either didn't perform in the final at all or where it has actually won the competition.

In [ ]:
my_cmap = plt.get_cmap("magma_r")
rescale = lambda y: y / 26

def plot_country_history(country, ax, df_plot):
    df_country_entries = (df_plot
                          .loc[df_plot['to_country'] == country]
                          .sort_values('year', ascending=False)
                          )
    country_name = df_country_entries['to_country'].iloc[0].title()

    # Add in NaN entries for years where the country did not perform in the finals
    present_years = df_country_entries['year'].unique()
    absent_years = list(set(range(1998, 2023)) - set(present_years))
    df_absent_entries = pd.DataFrame(dict(year=absent_years, total_points=np.nan,
                        rank= np.nan, to_country=country))
    df_country_entries = pd.concat([df_country_entries, df_absent_entries], ignore_index=True)

    # Plot data
    ax.bar(df_country_entries['year'], df_country_entries['total_points'],
            color=my_cmap(rescale(df_country_entries['rank'])))

    # Annotate absent entries, with a special marker for 2020 (cancelled)
    for year in absent_years:
        marker = "_" if year == 2020 else "x"
        ax.scatter(x=year, y=0.02, s=25, color='grey', marker=marker, transform=ax.get_xaxis_transform())

    # Annotate winning entries
    winning_entries = df_country_entries.loc[df_country_entries['rank'] == 1, ['year', 'total_points']].drop_duplicates()
    if winning_entries.shape[0] > 0:
        ax.scatter(x=winning_entries['year'], y=winning_entries['total_points'], s=50, color='gold', marker="*", edgecolor='black', zorder=5)

    ax.set_xlim(1997, 2023)
    ax.set_title(country_name)


Before looking at the top performers, let's have a look at the UK 🇬🇧 first. Every couple of years, the UK has a good year with a top-ten position, but hasn't come first since 1997 with [Katrina and the Waves](https://www.youtube.com/watch?v=azw4Kh8Rqpw) (not in our dataset). 2022 was the most successful year since then, with Sam Ryder becoming second with the song [Space Man](https://www.youtube.com/watch?v=RZ0hqX_92zI).

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 4), sharey=True, sharex=True, squeeze=True)

plot_country_history('united kingdom', ax, df)

legend_elements = [Line2D([0], [0], marker='*', color='white', label='Winner',
                          markerfacecolor='gold', markersize=12, markeredgecolor='black'),                          
                  Line2D([0], [0], marker='x', color='white', label='Did not perform in final',
                          markerfacecolor='grey', markersize=8, markeredgecolor='grey'), 
                  Line2D([0], [0], marker='_', color='white', label='Competition cancelled',
                          markerfacecolor='grey', markersize=8, markeredgecolor='grey')]
fig.legend(handles=legend_elements, loc='right', ncol=1, bbox_to_anchor=(0.9, -0.05))

sm = plt.cm.ScalarMappable(cmap=my_cmap, norm=plt.Normalize(vmin=1, vmax=26))
cbaxes = fig.add_axes([0.2, -0.05, 0.2, 0.05]) # x y deltax deltay

fig.colorbar(sm, ax=ax, orientation='horizontal', fraction=0.02, pad=0.1, label='Position', 
             cax = cbaxes)

plt.show()

Compared to the UK, how do the _most successful_ countries perform over the years? We'll have a look at the top four countries, for both the total and the average scores.

In [ ]:
fig = plt.figure(layout='constrained', figsize=(12, 6))
subfigs = fig.subfigures(2, 1, wspace=0.1)

for subfig, country_group, title in zip(subfigs,
                                        [multiple_winners, highest_scorers],
                                        ['Multiple winners', 'Countries with highest average points']
                                        ):
    axs = subfig.subplots(1, 4, sharey=True, sharex=True, squeeze=True)
    subfig.suptitle(title, fontsize=14, fontweight='bold')
    for i, country in enumerate(country_group.index[:4]): 
        plot_country_history(country, axs[i], df)
        if country == 'Yugoslavia':
            axs[i].set_title("Yugoslavia\nor, Serbia and Montenegro")

# Add in colorbar and legend
sm = plt.cm.ScalarMappable(cmap=my_cmap, norm=plt.Normalize(vmin=1, vmax=26))
cbaxes = fig.add_axes([0.3, -0.08, 0.2, 0.05]) # x y deltax deltay
fig.colorbar(sm, ax=ax, orientation='horizontal', fraction=0.02, pad=0.1, label='Position', cax=cbaxes)
fig.legend(handles=legend_elements, loc='right', ncol=1, bbox_to_anchor=(0.7, -0.08))

plt.show()

There's a clear difference here to the picture of the UK.
Ukraine 🇺🇦 and Sweden 🇸🇪 have not only won three times each, but have also placed within the top ten many times over the years (indicated by the light bars).
Some countries haven't participated very often, but have done really well the few times they did, such as Bulgaria 🇧🇬.
This is reflected in their second-place ranking in the table of average scores.

## Voting patterns between country pairs

There is a lot more to explore in the Eurovision data. Countries are performing but are also voting for each other. Do countries mainly vote for the actual performance or are the some general biases between countries? To dig deeper into this, we'll explore voting patterns between country pairs.

As we've seen above, some countries just perform better on average than others. To look into potential country-pair voting biases, we therefore look for deviations from a countries average performance score. This is basically asking the question: Does a certain country vote better / worse for another country compared to all other countries?

We start by calculating the voting deviations from the mean each country receives. 

In [ ]:
df_voting= df[['from_country', 'points', 'to_country', 'year']].copy()

# The average votes for each To country
df_voting['avg_votes_per_pcountry'] = df_voting.groupby(['to_country'])['points'].transform('mean')

# The average votes for each To country
df_voting['avg_votes_per_pcountry_per country'] = df_voting.groupby(['to_country', 'from_country'])['points'].transform('mean')

# For each voting instance calculate the deviation from the average votes
df_voting['avg_difference_votes'] = df_voting['avg_votes_per_pcountry_per country'] - df_voting['avg_votes_per_pcountry']    

# If no_instances < 3, set avg_difference_votes to NaN, since this is not a reflective sample
df_voting['no_instances'] = df_voting.groupby(['from_country', 'to_country'])['avg_difference_votes'].transform('count')
df_voting.loc[df_voting['no_instances'] < 3, 'avg_difference_votes'] = np.nan

df_voting.head()

Before plotting the voting deviations, we have a look at the average score each country receives from each other country. The heatmap shows lighter colours for higher scores.

In [ ]:
df_voting_heatmap = df_voting[['from_country', 'avg_votes_per_pcountry_per country', 'to_country']]
df_voting_heatmap = df_voting_heatmap.drop_duplicates()

# if from country = to country, set avg_difference_votes to NaN
df_voting_heatmap.loc[df_voting_heatmap['from_country'] == df_voting_heatmap['to_country'], 'avg_votes_per_pcountry_per country'] = np.nan

df_voting_heatmap = df_voting_heatmap.dropna(subset=['avg_votes_per_pcountry_per country'])

df_heatmap = df_voting_heatmap.pivot(index='from_country', columns='to_country', values='avg_votes_per_pcountry_per country')

df_heatmap.head()

import scipy.spatial as sp, scipy.cluster.hierarchy as hc

df_heatmap2 = df_heatmap.copy()

# row order to match column order
df_heatmap2 = df_heatmap2.reindex(df_heatmap2.columns)

row_dism = 1 - df_heatmap2.T.corr()
row_linkage = hc.linkage(sp.distance.squareform(row_dism), method='complete')

plot = sns.clustermap(df_heatmap2, row_linkage=row_linkage, col_linkage=row_linkage, 
                figsize=(9,8), mask=df_heatmap2.isnull(), 
                dendrogram_ratio= [0.15, 0.01],
                cbar_pos = (0.85,1,0.1,.019), # x,y, delta x, delta y
                cbar_kws={"orientation": "horizontal"},
                #cmap='viridis', 
                #center=0, 
                xticklabels=1, yticklabels=1)

plot.ax_col_dendrogram.set_visible(False) 

from matplotlib.patches import Rectangle
ax = plot.ax_heatmap


plot.fig.suptitle('Average score received from voting counties', fontsize=16, y=1.05)
plot.ax_heatmap.set_ylabel('Voting country')
plot.ax_heatmap.set_xlabel('Performing country')

plt.show()

From this plot, we can see a few patterns emerging. Some countries tend to vote higher for each other, forming lighter squares in the heatmap. One such square in the bottom right is around Slovenia 🇸🇮, Bosnia and Herzegovina 🇧🇦, Croatia 🇭🇷, Serbia 🇷🇸, which all give each other relatively high votes on average. But some countries will just genuinely perform well. To disentangle these two factors, we can visualise how each country deviates from the average vote for another country. The next heatmap takes a bit more preparation:

In [ ]:
df_voting_heatmap = df_voting[['from_country', 'avg_difference_votes', 'to_country']]
df_voting_heatmap = df_voting_heatmap.drop_duplicates()

# if from country = to country, set avg_difference_votes to NaN
df_voting_heatmap.loc[df_voting_heatmap['from_country'] == df_voting_heatmap['to_country'], 'avg_difference_votes'] = np.nan

df_voting_heatmap = df_voting_heatmap.dropna(subset=['avg_difference_votes'])

df_heatmap = df_voting_heatmap.pivot(index='from_country', columns='to_country', values='avg_difference_votes')

df_heatmap.head()

In [ ]:
import scipy.spatial as sp, scipy.cluster.hierarchy as hc

df_heatmap2 = df_heatmap.copy()

# row order to match column order
df_heatmap2 = df_heatmap2.reindex(df_heatmap2.columns)

row_dism = 1 - df_heatmap2.T.corr()
row_linkage = hc.linkage(sp.distance.squareform(row_dism), method='complete')

plot = sns.clustermap(df_heatmap2, row_linkage=row_linkage, col_linkage=row_linkage, 
                figsize=(9,8), mask=df_heatmap2.isnull(), 
                dendrogram_ratio= [0.15, 0.01],
                cbar_pos = (0.85,1,0.1,.019), # x,y, delta x, delta y
                cbar_kws={"orientation": "horizontal"},
                cmap='RdBu_r', center=0, xticklabels=1, yticklabels=1)

plot.ax_col_dendrogram.set_visible(False) 

from matplotlib.patches import Rectangle
ax = plot.ax_heatmap

ax.add_patch(Rectangle((4, 4), 9, 9, fill=False, edgecolor='black', lw=2))
ax.add_patch(Rectangle((df_heatmap2.shape[1]-13, df_heatmap2.shape[1]-13), 13, 13, fill=False, edgecolor='black', lw=2))
ax.add_patch(Rectangle((13, 13), 7, 7, fill=False, edgecolor='black', lw=2))

ax.add_patch(Rectangle((0, 27), 1, 4, fill=False, edgecolor='cornflowerblue', lw=2)) # turkey-germany, netherlands, belgium, france
ax.add_patch(Rectangle((27, 0), 4, 1, fill=False, edgecolor='cornflowerblue', lw=2))

ax.add_patch(Rectangle((20, 25), 4, 1, fill=False, edgecolor='darkcyan', lw=2)) # italy-malta, san marino, albania, portugal
ax.add_patch(Rectangle((25, 20), 1, 4, fill=False, edgecolor='darkcyan', lw=2))

ax.add_patch(Rectangle((13, 38), 1, 1, fill=False, edgecolor='indigo', lw=2)) # serbia-hungary
ax.add_patch(Rectangle((38, 13), 1, 1, fill=False, edgecolor='indigo', lw=2))

ax.add_patch(Rectangle((21, 17), 1, 1, fill=False, edgecolor='springgreen', lw=2)) # north-macedonia-albania
ax.add_patch(Rectangle((17, 21), 1, 1, fill=False, edgecolor='springgreen', lw=2))

ax.add_patch(Rectangle((30, 23), 1, 1, fill=False, edgecolor='yellow', lw=2)) # portugal-france
ax.add_patch(Rectangle((23, 30), 1, 1, fill=False, edgecolor='yellow', lw=2))

plot.fig.suptitle('Average deviation from mean performance vote', fontsize=16, y=1.05)
plot.ax_heatmap.set_ylabel('Voting country')
plot.ax_heatmap.set_xlabel('Performing country')

plt.show()

In this plot, countries are grouped using hierarchical clustering, ordering countries with similar voting patterns together. The heatmap shows the voting deviations from the mean each country receives. Lighter colours indicate higher deviations from the mean. 

There are a number of interesting details in this plot highlighted by different coloured boxes.  

For example, the small blue boxes on the left indicate that Turkey's entries have consistently obtained higher-than-average scores from Germany, the Netherlands, Belgium, and France; but that this isn't reciprocated, as the other blue box on the top shows.
A possible hypothesis for this may be the fact there is a significant Turkish diaspora in these countries: according to Pashayan (2012)*, there were 10 million "Euro-Turks" in these four countries in 2012. This suggests that migration data between two countries might be another useful thing to look at.

The large black boxes show clusters of countries who vote highly for each other and tend to be closer geographically. For example, the cluster in the bottom right contains many Nordic countries, the box in the middle balkan countries and the top box ex-soviet countries. overall, this suggests that sharing a border and being close geographically matters for how countries vote.

* REF (to be done up properly, and actually just ref 5 from https://en.wikipedia.org/wiki/Turkish_diaspora): Pashayan, Araks (2012), "Integration of Muslims in Europe and the Gülen", in Weller, Paul; Ihsan, Yilmaz (eds.), European Muslims, Civility and Public Life: Perspectives On and From the Gülen Movement, Continuum International Publishing Group, ISBN 978-1-4411-0207-2.

## Voting Pair Plots

We can also look at these country relationships using scatterplots. In the following plots, each point is an average voting score from one country to another.

In [ ]:
countries = df['from_country'].unique()

# Create a new dataframe containing the data for the scatter plot
votes = pd.DataFrame({'performer': [], 'voter': [], 'times_competed': [], 'times_voted': [], 'total_points': [], 'average_points': []})

# Compare every country with every other country
for performer in countries:
    times_competed = len(df.loc[df['to_country'] == performer]['year'].unique())
    for voter in countries:
        times_voted = len(df.loc[(df['to_country'] == performer) & (df['from_country'] == voter) & (df['points'] > 0)])
        total_points = df.loc[(df['to_country'] == performer) & (df['from_country'] == voter)]['points'].sum()
        average_points = df.loc[(df['to_country'] == performer) & (df['points'] >= 0) & (df['from_country'] == voter)]['points'].mean()
        votes.loc[len(votes)] = [performer, voter, times_competed, times_voted, total_points, average_points]
print('Check every country is matched with very other: {}'.format(len(countries)**2 == len(votes)))

# Add the average points received overall for each country
votes = votes.merge(df.groupby('to_country')['points'].mean().reset_index(), 
    how='left', left_on='performer', right_on='to_country')

# Calculate the voting country's deviation from the average points given
votes.rename(columns={'points': 'overall_avgerage_points'}, inplace=True)
votes['deviation_from_average'] = votes['average_points'] - votes['overall_avgerage_points']

In [ ]:
# Render a basic static scatter plot
fig = plt.figure(figsize=(8, 8), dpi=90)
votes.plot.scatter(x='total_points', y='average_points', alpha=0.5, ax = plt.gca())

plt.show()

To better explore the patterns yourself, we made an interactive visualisation. Hovering over the datapoints will show who voted for whom how often and what the average and total scores were. The left upper corner shows where countries have only voted a few times for another country, but given them very high points. For example, the top point in the left upper corner reveals that Serbia 🇷🇸 has only voted for Montenegro 🇲🇪 once, but given them 12 points, the highest score.

In [ ]:
import plotly.express as px

# Render an interactive (hoverable) plot
fig = px.scatter(votes, x='total_points', y='average_points')
fig.update_traces(hovertemplate='Performer: %{customdata[0]}'
                  + '<br>Voter: %{customdata[1]}'
                  + '<br>Total Eurovisions competed: %{customdata[2]}'
                  + '<br>Total times voted for by selected country: %{customdata[3]}'
                  + '<br>Total points given: %{customdata[4]}'
                  + '<br>Average points: %{customdata[5]:.2f}',
                  customdata=votes,
                  marker={'color': 'rgba(50, 50, 150, 0.1)', 'opacity': 0.5, 'size': 6,
                          'line': {'color': 'rgba(50, 50, 150, 1.0)', 'width': 1}})
fig.update_layout(hoverlabel_align='left', width=640, height=640, margin=dict(l=20, r=20, t=20, b=20),
                  xaxis={'title': 'Total voter points given to specific country'},
                  yaxis={'title': 'Average points'})
fig.show()

In [ ]:
# Plotly plot of performing countries on the y-axis and average votes received on the x-axis
votes = votes.sort_values(by='average_points', ascending=False)

fig = px.scatter(votes, x='average_points', y='performer', color='deviation_from_average', 
    color_continuous_scale=px.colors.diverging.RdYlGn,
    color_continuous_midpoint=0
)
fig.update_traces(hovertemplate='Performer: %{customdata[0]}'
    + '<br>Voter: %{customdata[1]}'
    + '<br>Total Eurovisions competed: %{customdata[2]}'
    + '<br>Total times voted for by selected country: %{customdata[3]}'
    + '<br>Total points given: %{customdata[4]}'
    + '<br>Average points by %{customdata[1]}: %{customdata[5]:.2f}'
    + '<br>Average points overall: %{customdata[7]:.2f}',
    customdata=votes)
fig.update_layout(hoverlabel_align='left', width=640, height=640, margin=dict(l=20, r=20, t=20, b=20),
    xaxis={'title': 'Average points received by selected country'},
    yaxis={'title': 'Performing Country'}, 
    coloraxis_colorbar=dict(title='Deviation from average points'))

fig.update_yaxes(tickfont_size=8)

fig.show()

## Exploring Country Friendships, Biases, and One-sided relationships

After looking at broader patterns, let's now dive into some specific country-pairs. 
We will look for the top country pairs which
* give each other very high scores
* give each other very low scores
* have a one-sided relationship, where one country gives high scores to another, but gets low scores back 

We'll filter out countries that haven't participated often enough to get a more robust estimate of the average voting score.

In [ ]:
# how often did each country participate?
n_participations = (df
                    .groupby('to_country')['year']
                    .nunique()
                    .sort_values(ascending=True))
# filter countries with less than 5 participations
countries_to_remove = n_participations[n_participations < 5].index.tolist()
# remove from df
df_pairs = df[~df['to_country'].isin(countries_to_remove)]
# get random 5 rows from df_pairs
# df_pairs.sample(5)

To look at more specific relationships, we'll calculate for each country pair how much their average votes for each differ. This value would be small if they both tend to give each other high or low scores, but it would be large if one country gives the other high scores, but gets low scores back. To check whether this worked, we have a look at the top five country pairs that give each other high votes.

In [ ]:
# Grouping by performer and voter, calculate mean votes and count number of years
df_pairs = (df_pairs
    .groupby(['from_country', 'to_country'])
    .agg(votes = ('points', 'mean'), num_years = ('year', 'count'))
    .reset_index()
    .sort_values('votes', ascending=False)
)

# Merge original dataframe with its reverse
df_pairs = (df_pairs
    .merge(df_pairs.rename(columns={'from_country': 'to_country', 
                               'to_country': 'from_country'}), 
           on=['from_country', 'to_country'])
    .drop_duplicates()
    .query('from_country != to_country')
    .assign(votes_diff = lambda x: abs(x['votes_x'] - x['votes_y']))
    .reindex(columns=['from_country', 'to_country', 'num_years', 'votes_x', 'votes_y', 'votes_diff'])
)

# create combined column with pairs in alphabetical order
df_pairs['country_pair'] = df_pairs[['from_country', 'to_country']].apply(lambda x: ' - '.join(sorted(x)), axis=1)
# Remove duplicate country pairs and the temporary country_pair column
df_pairs = (df_pairs
            .drop_duplicates(subset=['country_pair'])
            .drop(columns=['country_pair']))

df_pairs.head()

To prepare the data for plotting, we filter for the top five country pairs that give each other high, low and unequal votes.

In [ ]:
# Number of country pairs to show for each category
N = 5

# top 5 country pairs with high votes for each other
top_highs = (df_pairs
    .query('votes_diff < 3')
    .sort_values('votes_x', ascending=False).head(N)
)

# top 5 country pairs with low votes for each other
top_lows = (df_pairs
    .query('votes_diff < 3')
    .sort_values('votes_x', ascending=True).head(N)
)

# top 5 unbalanced country pairs (one gives high votes to the other, but gets low votes back)
top_one_sided = (df_pairs
            .sort_values('votes_diff', ascending=False).head(N))

# combine 
top_relationships = (pd.concat([top_highs, top_lows, top_one_sided]))
# add grouping
top_relationships['group'] = ['high'] * N + ['low'] * N + ['one-sided'] * N

# sort by votes_x
top_relationships = top_relationships.sort_values('votes_x')

top_relationships['y'] = range(1, len(top_relationships) + 1)

top_relationships

In [ ]:
# Replace group names with numeric values for sorting
group_order = {'low': 1, 'one-sided': 2, 'high': 3}
top_relationships['group_order'] = top_relationships['group'].replace(group_order)

# Create a new column representing the minimum value between 'votes_x' and 'votes_y'
top_relationships['min_votes'] = top_relationships[['votes_x', 'votes_y']].min(axis=1)

# Sort the DataFrame by 'group_order' and 'min_votes'
top_relationships = top_relationships.sort_values(by=['group_order', 'min_votes'], ascending=[True, True])

# Reassign the 'y' column after sorting
top_relationships['y'] = range(1, len(top_relationships) + 1)

Let's plot the results!

In [ ]:
import matplotlib.colors as mcolors

fig, ax = plt.subplots(figsize=(10, 6))
ax.grid(axis='x', linestyle='-', linewidth=0.5)

# create a color map with 6 colors from red to blue
colors = sns.color_palette("YlOrRd", 6)[::-1]
cmap = mcolors.LinearSegmentedColormap.from_list("custom", colors, N=len(colors))

# get color map with viridis colors for 6 bins
cmap = plt.get_cmap('viridis' )

#colors = ["#b2182b", "#d1e5f0", "#92c5de", "#4393c3", "#2166ac", "#053061" ]
colors = ["#d53e4f", "#e6f598", "#abdda4", "#66c2a5", "#3288bd", "#5e4fa2"]
cmap = mcolors.LinearSegmentedColormap.from_list("custom", colors, N=len(colors))


# define the range boundaries
bins = np.arange(0, 13, 2)

# add text labels and points
for i, row in top_relationships.iterrows():
    ax.text(row['votes_x']+0.2, row['y'] ,row['to_country'],  ha='left', va='center', fontsize=14)  # increase the value added to 'votes_x' for more space
    ax.text(row['votes_y']-0.2, row['y'] , row['from_country'],  ha='right', va='center', fontsize=14)  # increase the value subtracted from 'votes_y' for more space

    # plot points with colors based on x-value range
    x_color = np.digitize(row['votes_x'], bins=bins, right=True)-1
    y_color = np.digitize(row['votes_y'], bins=bins, right=True)-1
    ax.scatter(row['votes_x'], row['y'], color=cmap(x_color/6), s=100, alpha=1, zorder=3)
    ax.scatter(row['votes_y'], row['y'], color=cmap(y_color/6), s=100, alpha=1, zorder=3)

    # add line between countries with slightly shortened length
    line_buffer = 0.1
    ax.plot([row['votes_x'] + line_buffer, row['votes_y'] - line_buffer], [row['y'], row['y']], color='black', alpha=0.7, zorder=2, linewidth=0.7)

ax.set_xlabel('Average received vote of a country from its paired country', fontsize=14)
ax.set_ylabel('')

# Hide the y axis
ax.get_yaxis().set_visible(False)
ax.spines[['top', 'right', 'left']].set_visible(False)

# Add legend
markers = [plt.Line2D([0,0],[0,0], color=cmap(i/6), marker='o', linestyle='') for i in range(6)]
plt.legend(markers, ['{}-{}'.format(i, i+2) for i in bins[:-1]], numpoints=1, loc='lower right', title='Vote ranges')

plt.title('Country pairs with high, low, and unequal votes for each other', fontsize=18,  loc='center', pad=15)

plt.show()

The figure shows the top five country pairs which give each other high, low and unequal scores.
The point next to a country name is the score it *received* from the other country. For example, in line with the previous figures, both France 🇫🇷 and Germany 🇩🇪 on average receive low scores from Turkey 🇹🇷, but vote highly for Turkey.
Cyprus 🇨🇾  and Greece 🇬🇷  give each other high scores, and Albania 🇦🇱 and Lithuania 🇱🇹 tend to give each other low scores.

The plot differs a bit from the results above, because it is looking at raw votes, not deviations from the mean vote.
Therefore, one-sided voting does not necessarily indicate bias in voting: it may simply be because the country has historically performed very well (or very poorly!) in the Eurovision Song Contest.

# Migration Data

 **TODO** Maybe plot prop_emigrants versus the score deviation?

One of our main goals was to predict the winner of the 2023 contest. On a finer scale, this means predicting how each country will vote for each other country. Arguably, countries with a large immigrant population from another country might give more favorable votes to that country. To test and incorporate this in our models, we used migration data from [Our World in Data](https://ourworldindata.org/migration). Specifically, we were interested in the proportion of immigrants in each country from each other country. We calculated both sides, i.e. proportion emigrants from the voter country in the perfomer country (v2p) and the other way round.

In [ ]:
# migration data
df.filter(like='migr').head()

In [ ]:
df_migration = df[['year', 'from_code2', 'to_code2', 'prop_emigrants_v2p', 'prop_emigrants_p2v', 'points']].copy()

# Calculate score deviation per year
df_migration['avg_votes'] = df_migration.groupby(['to_code2', 'year'])['points'].transform('mean')
df_migration['deviation'] = df_migration['points'] - df_migration['avg_votes']


In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True, figsize=(12, 6))
sns.regplot(x='prop_emigrants_v2p', y='deviation', data=df_migration, ax=ax1)
sns.regplot(x='prop_emigrants_p2v', y='deviation', data=df_migration, ax=ax2)

ax1.set_xlabel('Proportion of emigrants from voting country in performing country')
ax2.set_xlabel('Proportion of emigrants from performing country in voting country')

plt.show()

In [ ]:
sns.lmplot(x='prop_emigrants_p2v', y='prop_emigrants_v2p', data=df_migration, scatter_kws={"s": 4})

# log scale axes
plt.xscale('log')
plt.yscale('log')

plt.xlabel("Proportion of emigrants from voting country in performing country")
plt.ylabel("Proportion of emigrants from performing country in voting country")
plt.show()

# Language Data

We are interested in three things:
1. Does the song contain english lyrics: yes vs no &#x2705;
2. What are the average votes for songs with english lyrics vs non-english lyrics &#x2705;
3. Does the performer sing in their official language yes vs no (english) vs no (non-english) &#x2705;
4. How many languages appear in the song vs votes? (can you hedge your bets?) &#x2705;
5. Are the votes received affected by whether or not a song is sung in the voting country's official language? &#x2705;

**TODO** Get language data cleaned up, see https://github.com/KatrionaGoldmann/Eurovision_TDS/blob/issue-5-exploratory_visualisations/eurovision/notebooks/language_visualistaions.ipynb.

**TODO** Discussion of Eurovision language rules (this has changed over time).

In this dataset, we have categorised the song language into one or more of: the country's official language (`own`), English (`eng`), or an entirely separate language (`other`).
When English *is* one of the country's official languages, both `own` and `other` are selected.
We can use this data to investigate how likely each country is to perform in English:

In [ ]:
df_performance = df[['year', 'Artist', 'to_country', 
       'total_points', 
       'rank', 'to_code2',
       'Official_languages', 'Language_sung', 'Contains_English',
       'Contains_NonEnglish', 'Contains_Multiple_Languages',
       'Number_of_Languages', 'Contains_Own_Language', 'gender']].drop_duplicates()

In [ ]:
df_performance.loc[df_performance['year'] == 2018]['to_country'].value_counts()

In [ ]:
# get the percentage of performances who contain own language
print('Percentage of performances in own language:', round(sum(df_performance['Contains_Own_Language'])/len(df_performance) *100), '%')
print('Percentage of performances in English:', round(sum(df_performance['Contains_English'])/len(df_performance) *100), '%')

print('Percentage of performances by gender:')
df_performance['gender'].value_counts()/len(df_performance) * 100

In [ ]:
df_language = df_performance.copy()

df_performance['English_only'] = (df_performance['Contains_English']) & (df_performance['Number_of_Languages'] == 1 )
df_performance['No_English'] = ~df_performance['Contains_English'] 
df_performance['Some_English'] = (df_performance['Contains_English']) & (df_performance['Number_of_Languages'] > 1 )

# for each country get the ratio of songs that contain only English, some English and no English
# then sort by the ratio of songs that contain only English
df_language = df_performance.groupby('to_country').agg({'English_only': 'mean', 'Some_English': 'mean', 'No_English': 'mean'})

df_language = df_language.sort_values(by=['English_only', 'Some_English'], ascending=True)

In [ ]:
colours = {"English_only":'royalblue', "No_English":'seagreen', "Some_English":'goldenrod'}

df_language.plot(kind='bar', figsize=(15, 6), stacked=True, color=colours)

plt.legend(['English only', 'Partly English', 'No English'], title="Performance languages", loc=[1, 1], 
        fontsize=14,  bbox_to_anchor=(0.51, 0., 0.5, 0.5), title_fontsize=16)

plt.title('How frequently countries sing in English', fontsize=20, pad=30)

plt.text(df_language.shape[0]-1, 1.1, 'Countries more likely to sing in English →', ha='right', va='center', fontsize=14)
plt.text(0.05, 1.1, '← Countries less likely to sing in English ', ha='left', va='center',  fontsize=14)
plt.xticks(rotation=-45, ha='left')
plt.xlabel('')
plt.ylabel('Ratio of performances')

plt.show()

In [ ]:
# convert wide to long format
df_long = df_performance[['English_only', 'No_English',	'Some_English', 'total_points']]

df_long = df_long.melt(id_vars=['total_points'], var_name='language', value_name='contains_language')

df_long['contains_language'] = df_long['contains_language'].astype(int)

In [ ]:
import scipy.stats as stats
from statannot import add_stat_annotation

# boxplots for each language type
ax = sns.boxplot(x='language', y='total_points', 
    data=df_long.loc[df_long['contains_language'] > 0], 
    palette=colours, showfliers=False, 
    order=['English_only', 'Some_English', 'No_English'])
sns.stripplot(x='language', y='total_points', 
    order=['English_only', 'Some_English', 'No_English'],
    data=df_long.loc[df_long['contains_language'] > 0], 
    jitter=0.25, size=2, color=".3", linewidth=0)

plt.title('Average points for performances in different languages', fontsize=20, pad=30)
plt.xlabel('')
plt.ylabel('Total points')

add_stat_annotation(ax, data=df_long.loc[df_long['contains_language'] > 0],
                    x='language', y='total_points', 
                    order=['English_only', 'Some_English', 'No_English'],
                    box_pairs=[("English_only", "No_English")],
                    test='Mann-Whitney', text_format='star', verbose=0)

plt.show()

In [ ]:
df_language = df_performance.copy()

df_performance['Own_language'] = (df_performance['Contains_Own_Language']) 
df_performance['Other_language'] = ~df_performance['Contains_Own_Language'] 

# for each country get the ratio of songs that contain only English, some English and no English
# then sort by the ratio of songs that contain only English
df_language = df_performance.groupby('to_country').agg({'Other_language': 'mean', 'Own_language': 'mean'})

df_language = df_language.sort_values(by=['Own_language', 'Other_language'], ascending=True)

In [ ]:
colours = {"Own_language":'teal', "Other_language":'lightsteelblue'}

df_language.plot(kind='bar', figsize=(15, 6), stacked=True, color=colours)

plt.legend(['Other language', 'Own language'], title="Performance languages", loc=[1, 1], 
        fontsize=14,  bbox_to_anchor=(0.51, 0., 0.5, 0.5), title_fontsize=16)

plt.title('How frequently countries sing in their official languages', fontsize=20, pad=30)

plt.text(df_language.shape[0]-1, 1.1, 'Countries more likely to sing in Official language →', ha='right', va='center', fontsize=14)
plt.text(0.05, 1.1, '← Countries less likely to sing in Official language ', ha='left', va='center',  fontsize=14)
plt.xticks(rotation=-45, ha='left')
plt.xlabel('')
plt.ylabel('Ratio of performances')

plt.show()

Typically those on the left of this plot are countries who sing in english but english is not their official language. Those on the right are countries who sing in their official language exclusively.

In [ ]:
sns.boxplot(x='Number_of_Languages', y='total_points', data=df_performance, showfliers=False)
sns.stripplot(x='Number_of_Languages', y='total_points', data=df_performance, jitter=0.25, size=2, color=".3", linewidth=0)

plt.title('Points Scored by Number of Languages appearing in Song', fontsize=20, pad=30)
plt.xlabel('Number of Languages')
plt.ylabel('Total points')


plt.show()

In [ ]:
# violin plot
sns.violinplot(x='Contains_Voting_Language', y='points', data=df, inner=None, color="cornflowerblue")

plt.show()

# Gender Data

In [ ]:
from matplotlib.colors import to_rgba

gender_df = df[['year', 'to_country', 'gender']].copy().drop_duplicates()

countries = gender_df.sort_values(by=['to_country'])['to_country'].drop_duplicates().to_list()
years = gender_df['year'].drop_duplicates().to_list()

gender_colours = {
    'female': to_rgba('seagreen'),
    'group': to_rgba('wheat'),
    'male': to_rgba('indianred'),
    'none': to_rgba('white')
}

gender = []
for year in years:
    missing = pd.DataFrame({'year': [year] * len(countries), 'to_country': countries, 'gender': ['none'] * len(countries)}).set_index('to_country')
    missing.update(gender_df[gender_df['year'] == year].set_index('to_country'))
    gender.append(list(map(lambda x: gender_colours[x], missing.sort_values(by=['to_country'])['gender'].to_list())))

gender = list(map(list, zip(*gender)))
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(10, 10)
img = plt.imshow(gender, aspect=0.7)
ax.set_xticks(range(len(years)))
ax.set_xticklabels(years)
plt.xticks(rotation=90)
ax.set_yticks(range(len(countries)))
ax.set_yticklabels(list(map(str.title, countries)))
ax.grid(False)
plt.title('Gender of Artists at Eurovision by Year and Performing Country\n(countries ordered alphabetically)')

plt.show()

In [ ]:
for gender in ['female', 'male', 'group']:
    count = len(gender_df[gender_df['gender'] == gender])
    print('Total {} performers: {} ({:.0%})'.format(gender, count, count/len(gender_df)))

def ratio(gender, x): return len([a for a in x if a == gender]) / len(x)

gender_df = gender_df.groupby('to_country').agg(
    female=pd.NamedAgg(column='gender', aggfunc=lambda x: ratio('female', x)),
    group=pd.NamedAgg(column='gender', aggfunc=lambda x: ratio('group', x)),
    male=pd.NamedAgg(column='gender', aggfunc=lambda x: ratio('male', x)),
)
gender_df = gender_df.sort_values(by=['female', 'group'], ascending=False)

gender_df.head()

In [ ]:
gender_colours = {
    'female': to_rgba('seagreen'),
    'group': to_rgba('wheat'),
    'male': to_rgba('indianred')#,
    #'none': to_rgba('white')
}

gender_df[gender_colours.keys()].plot(kind='bar', figsize=(15, 6), stacked=True, color=gender_colours)
plt.text(gender_df.shape[0] - 1, 1.1, 'Countries less likely to pitch female singers →', ha='right', va='center', fontsize=12)
plt.text(0.05, 1.1, '← Countries more likely to pitch female singers', ha='left', va='center',  fontsize=12)
plt.legend(['Female', 'Group', 'Male'], title="Performers' gender", loc=[1, 1], fontsize=8,  bbox_to_anchor=(0.51, 0., 0.5, 0.5))
plt.title('Gender of Artists at Eurovision by Performing Country', fontsize=16, pad=30)
plt.xticks(rotation=-45, ha='left')
plt.xlabel('')
plt.ylabel('Ratio of performances')

plt.show()

In [ ]:
gender_df = df[['year', 'to_code2', 'gender']].copy().drop_duplicates()

def ratio(gender, x): return len([a for a in x if a == gender]) / len(x)

gender_df = gender_df.groupby('year').agg(
    female=pd.NamedAgg(column='gender', aggfunc=lambda x: ratio('female', x)),
    group=pd.NamedAgg(column='gender', aggfunc=lambda x: ratio('group', x)),
    male=pd.NamedAgg(column='gender', aggfunc=lambda x: ratio('male', x)),
)

gender_df.head()

In [ ]:
# subset gender_colours to genders in gender_df columns
# gender_colours = gender_colours[gender_df.columns.to_list()]

gender_df.plot(kind='bar', figsize=(15, 6), stacked=True, color=gender_colours)

plt.legend(['Female', 'Group', 'Male'], title="Performers' gender", loc=[1, 1], fontsize=8,  bbox_to_anchor=(0.51, 0., 0.5, 0.5))
plt.title('Gender of Artists at Eurovision by Year', fontsize=16, pad=30)
plt.xticks(rotation=-45, ha='left')
plt.xlabel('')
plt.ylabel('Ratio of performances')

plt.show()

In [ ]:
gender_df = df[['year', 'to_code2', 'gender', 'points']].copy()
genders = list(gender_colours.keys())[:3]
votes = []
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1)
for gender in genders:
    votes_df = gender_df[gender_df['gender'] == gender]['points']
    votes.append(votes_df)
    plt.axvline(votes_df.mean(), color=gender_colours[gender], linestyle='dashed', linewidth=1)

plt.hist(votes, density=True, bins=range(14), color=list(gender_colours.values())[:3], label=[gender.title() for gender in genders], align='left')
plt.xlabel('Points awarded')
plt.ylabel('Frequency')
plt.title('Frequency of Points Awarded Categorised by Gender\n(Dashed line indicates mean score)')
plt.legend()
plt.xticks(range(13))
ax.set_xticklabels([str(i) for i in range(13)])

plt.show()

In [ ]:
# TODO: check if male get higher average votes. 

# Collective Visualisations

- Heatmap for correlation plot
- Martin's plot
- Geographical plots
- Scatter plots

In [ ]:
df_performance = df[['year', 'Artist', 'to_country', 
       'total_points', 'rank', 'to_code2', 
       'Official_languages', 'Language_sung', 'Contains_English',
       'Contains_NonEnglish', 'Contains_Multiple_Languages',
       'prop_emigrants_v2p', 'prop_emigrants_p2v', 'has_border',
       'comps_without_win',
       'Number_of_Languages', 'Contains_Own_Language', 'gender']].drop_duplicates()

In [ ]:
import matplotlib

# Define the variables of interest and their data types
vars_of_interest = {
    'Contains_Own_Language': 'binary',
    'Contains_Voting_Language': 'binary',
    'Contains_English': 'binary',
    'Contains_NonEnglish': 'binary',
    'prop_emigrants_v2p': 'numeric', 
    'prop_emigrants_p2v': 'numeric', 
    'has_border': 'binary',
    'gender': 'categorical', 
    'comps_without_win': 'numeric'
}

# Define the figure and axes
nc = int(np.ceil(len(vars_of_interest)/2))
fig, axes = plt.subplots(nrows=2, ncols=nc, figsize=(20, 10), sharey=True)

# Loop through the dictionary
for i, (key, value) in enumerate(vars_of_interest.items()):
    j, k = 0, i
    if i > (nc - 1):
        j, k = 1, i - nc

    if value == 'categorical':
        sns.violinplot(ax=axes[j, k], x=key, y='points', data=df, color='tab:blue', inner=None, showmeans=True)
    elif value == 'binary':
        sns.violinplot(ax=axes[j, k], x=key, y='points', data=df, color='tab:blue', inner=None, showmeans=True)
    else:
        sns.regplot(ax=axes[j, k], x=key, y='points', data=df, ci=95, 
                    color='tab:blue', scatter_kws={'alpha': 0.4, 'edgecolor': 'none', 's': 20}, 
                    line_kws={'color': 'tab:orange'})

        # if key contains Prop then log scale x axis
        if key.startswith('prop'):
            axes[j, k].set_xscale('log')
            axes[j, k].set_xticks([0.01, 0.1, 1, 10, 100])
            axes[j, k].get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
            axes[j, k].set_xlim(0.01, 1)

    axes[j, k].set_ylim(-1, 13)
    axes[j, k].set_xlabel(key.capitalize())
    axes[j, k].set_ylabel('Votes')
    axes[j, k].set_title(f'Votes vs. {key.capitalize()}')

fig.suptitle('Relationships between Covariates and Eurovision Votes', fontsize=16)

fig.tight_layout()
fig.subplots_adjust(hspace=0.25)

# Shuffle the bottom row along there are an odd number of variables
if len(vars_of_interest) % 2 == 1:
    axes[1][-1].remove()
    # distance between two axes
    if len(vars_of_interest) >= 3:
        dist = axes[0][1].get_position().x0 - axes[0][0].get_position().x0
        for ax in axes[1]:
            pos = ax.get_position()
            ax.set_position([pos.x0 + (dist / 2), pos.y0, pos.width, pos.height])

plt.show()

In [ ]:
# plot comps_without_win vs. points
sns.violinplot(x='points', y='comps_without_win', data=df)
# add jitter
sns.stripplot(x='points', y='comps_without_win', data=df, jitter=True, color='black', alpha=0.2)

plt.title('Votes vs. Number of Competitions without a Win')
plt.ylabel('Number of Competitions without a Win')
plt.xlabel('Votes')

plt.show()

In [ ]:
df[['points', 'comps_without_win']].corr()

In [ ]:
# One hoy encoding for the gender variable
df = pd.concat([df, pd.get_dummies(df['gender'])], axis=1)

# correlation plot of numeric and binary variables
df_corr = df[['points', 'rank',
       'total_points', 'Contains_English',
       'Contains_NonEnglish', 'Contains_Multiple_Languages',
       'Number_of_Languages', 'Contains_Own_Language', 'Contains_Voting_Language',
       'female', 'male', 'group',
       'prop_emigrants_v2p',  'prop_emigrants_p2v', 'has_border',
       'comps_without_win']].corr()

# heatmap of the correlation matrix
sns.heatmap(df_corr.corr(), cmap='coolwarm', annot=True, fmt='.2f', annot_kws={'fontsize': 7})

# replace _ in labels with space
labels = [label.replace('_', ' ').title() for label in df_corr.columns]
plt.xticks(np.arange(len(labels)), labels, rotation=-45, ha='left', va='top')
plt.yticks(np.arange(len(labels)) + 0.5, labels, rotation=0, va='center')

plt.show()

# Modelling

[...]

# Conclusions

Round everything up, draw conclusions.